In [241]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
import os
from datetime import datetime

In [242]:
layers = [512, 256, 64, 32, 10]
learning_rate = 0.01
n_epochs = 10001
BATCH_SIZE = 128
n_inputs = 28*28

In [243]:
tf.reset_default_graph()

train_data = pd.read_csv("train.csv")

# train_labels = pd.get_dummies(train_data.iloc[:, 0]).values
train_labels = train_data.iloc[:, 0].values
train_data = train_data.iloc[:, 1:].values

batch_size = tf.placeholder(tf.int64, name="batch_size")
X_train = tf.placeholder(tf.float64, shape=[None, n_inputs], name="X")
y_train = tf.placeholder(tf.int64, shape=[None], name="y")

def map_fn(data, labels):
    data = tf.math.divide(data, 255)
    return data, labels

dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train)).repeat().shuffle(len(train_data))
dataset = dataset.batch(batch_size)
dataset = dataset.map(map_fn, num_parallel_calls = 6)
dataset = dataset.prefetch(1)

iter = dataset.make_initializable_iterator()
data, labels = iter.get_next()

In [244]:
with tf.name_scope("dnn"):
#     input_x = keras.Input(data.get_shape().as_list())
#     X = layers.Dense(layers[0], activation="relu")(input_x)
#     X = layers.Dense(layers[1], activation="relu")(X)
#     X = layers.Dense(layers[2], activation="relu")(X)
#     X = layers.Dense(layers[3], activation="relu")(X)
#     logits = layers.Dense(layers[4], activation="softmax")(X)
    X = tf.layers.dense(data, layers[0], name="hidden1", activation=tf.nn.relu)
    X = tf.layers.dense(X, layers[1], name="hidden2", activation=tf.nn.relu)
    X = tf.layers.dense(X, layers[2], name="hidden3", activation=tf.nn.relu)
    X = tf.layers.dense(X, layers[3], name="hidden4", activation=tf.nn.relu)
    logits = tf.layers.dense(X, layers[4], name="outputs")
    logits = tf.cast(logits, tf.float32)

with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=labels, logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")
    loss_summary = tf.summary.scalar("log_loss", loss)

with tf.name_scope("train"):
    optimizer = tf.train.AdamOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)
    
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, labels, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float64), name="accuracy")
    accuracy_summary = tf.summary.scalar("log_accuracy", accuracy)

with tf.name_scope("init"):
    init = tf.global_variables_initializer()
    saver = tf.train.Saver()

In [245]:
def logdir(prefix=""):
    now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
    root_dir = "tf_logs/"
    return root_dir + prefix + "run-" + now

In [246]:
checkpoint_path = "/tmp/deep_mnist_dnn.ckpt"
checkpoint_epoch_path = checkpoint_path + ".epoch"
checkpoint_final_model = "./tmp/deep_mnist_dnn.ckpt"

file_writer = tf.summary.FileWriter(logdir("MNIST"), tf.get_default_graph())

with tf.Session() as sess:
    sess.run(iter.initializer, feed_dict={X_train: train_data, y_train: train_labels, batch_size: BATCH_SIZE})
    
    if os.path.isfile(checkpoint_epoch_path):
        with open(checkpoint_epoch_path, "rb") as f:
            start_epoch = int(f.read())
        print("Foi interrompido! Parou no {} epoch".format(start_epoch))
        saver.restore(sess, checkpoint_path)
        
    else:
        start_epoch = 0
        sess.run(init)
    
    for epoch in range(start_epoch, n_epochs):
        _, _loss_summary, _accuracy_summary = sess.run([training_op, loss_summary, accuracy_summary])
        
        if epoch % 10 == 0:
            file_writer.add_summary(_loss_summary, epoch)
            file_writer.add_summary(_accuracy_summary, epoch)
            saver.save(sess, checkpoint_path)
            with open(checkpoint_epoch_path, "wb") as f:
                f.write(b"%d" % (epoch + 1))
                
        if epoch % 200 == 0:
            loss_val = sess.run(loss)
            print("Loss:", loss_val)
    
    saver.save(sess, checkpoint_final_model)
    sess.run(iter.initializer, feed_dict={X_train: train_data, y_train: train_labels, batch_size: 42000})
    accuracy_val, data_val = sess.run([accuracy, data])
    print("Acurácia:", accuracy_val)
    
    os.remove(checkpoint_epoch_path)

KeyboardInterrupt: 

In [247]:
test_data = pd.read_csv("test.csv")
test_data = test_data.values

with tf.Session() as sess:
    saver.restore(sess, checkpoint_path)
    sess.run(iter.initializer, feed_dict={X_train: test_data, y_train: test_data[0], batch_size: len(test_data)})
    logits = sess.run(logits)
    y_pred = tf.argmax(logits, axis=1)

INFO:tensorflow:Restoring parameters from /tmp/deep_mnist_dnn.ckpt


InvalidArgumentError: All components must have the same size in the 0th dimension
	 [[node TensorSliceDataset (defined at <ipython-input-243-0decdd1aebe9>:22) ]]

Caused by op 'TensorSliceDataset', defined at:
  File "c:\users\luis_\appdata\local\programs\python\python37\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "c:\users\luis_\appdata\local\programs\python\python37\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "c:\users\luis_\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "c:\users\luis_\appdata\local\programs\python\python37\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "c:\users\luis_\appdata\local\programs\python\python37\lib\site-packages\ipykernel\kernelapp.py", line 505, in start
    self.io_loop.start()
  File "c:\users\luis_\appdata\local\programs\python\python37\li